# dataset.py, tester.py만 다시 넣으면됨

# 구글 마운트

In [ ]:
# prompt: 구글 마운트

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# drive에서 local로 복사

In [ ]:
# 1. Google Drive → Colab으로 압축 파일 복사
!cp "/content/drive/MyDrive/Colab Notebooks/APID/mimic-cxr.zip" /content/

# 2. 압축 해제 + 시간 측정
import time
start = time.time()
!unzip -q /content/mimic-cxr.zip -d /content/data/
print("압축 해제 완료:", round(time.time() - start, 2), "초")

# 3. 폴더별 이미지 수 출력
import os

base_path = '/content/data'
splits = ['train', 'valid', 'test']

for split in splits:
    split_path = os.path.join(base_path, split)
    num_images = len([f for f in os.listdir(split_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"{split} 이미지 수: {num_images}장")


압축 해제 완료: 25.27 초
train 이미지 수: 83642장
valid 이미지 수: 710장
test 이미지 수: 1446장


# R2Gen 깃에서 불러오는 코드

In [ ]:
!git clone https://github.com/zhjohnchan/R2Gen.git
%cd R2Gen
!ls

Cloning into 'R2Gen'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 83 (delta 19), reused 22 (delta 12), pack-reused 46 (from 1)
Receiving objects: 100% (83/83), 70.19 MiB | 39.75 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/R2Gen
compute_ce.py  main_train.py	  pycocoevalcap      train_iu_xray.sh
data	       models		  README.md	     train_mimic_cxr.sh
main_plot.py   modules		  test_iu_xray.sh
main_test.py   plot_mimic_cxr.sh  test_mimic_cxr.sh


# R2Gen 사용할떄 필요한 모델

In [ ]:
!pip install nltk
!pip install scikit-image
!pip install matplotlib
!pip install pycocotools
!pip install pandas
!pip install pillow
!pip install tqdm
!pip install numpy
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

# BLEU 계산을 위한 tokenizer

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# 25%, 50%, 75%, 100% 비율로 나눠서 json 파일만들고 기존 valid.json, test.json과 병합 하는 코드 1번만 실행

In [ ]:
import pandas as pd
import json
import os
import random

def convert_partial_csv_to_json(csv_path, image_dir, output_dir, split_name="train", ratios=[0.25, 0.5, 0.75, 1.0], seed=42):
    df = pd.read_csv(csv_path)
    total = len(df)
    print(f"총 데이터 수: {total}개")

    os.makedirs(output_dir, exist_ok=True)

    for ratio in ratios:
        n = int(total * ratio)
        df_sampled = df.sample(n=n, random_state=seed)

        items = []
        for _, row in df_sampled.iterrows():
            item = {
                "id": row["filename"],
                "report": row["text"],
                "image_path": os.path.join(image_dir, row["filename"])
            }
            items.append(item)

        json_data = {split_name: items}
        output_path = os.path.join(output_dir, f"train_{int(ratio*100)}.json")
        with open(output_path, 'w') as f:
            json.dump(json_data, f, indent=2)
        print(f"✅ 저장 완료: {output_path} ({n}개 샘플)")

def combine_split_jsons(train_json, val_json, test_json, save_path):
    with open(train_json, 'r') as f:
        train_data = json.load(f)
    with open(val_json, 'r') as f:
        val_data = json.load(f)
    with open(test_json, 'r') as f:
        test_data = json.load(f)

    train_items = train_data.get('train', train_data)
    val_items = val_data.get('val', val_data)
    test_items = test_data.get('test', test_data)

    combined = {
        "train": train_items,
        "val": val_items,
        "test": test_items
    }

    with open(save_path, 'w') as f:
        json.dump(combined, f, indent=2)
    print(f"✅ 병합 완료: {save_path} (train: {len(train_items)}, val: {len(val_items)}, test: {len(test_items)})")

# ===============================
# ✅ 경로 설정 및 실행
# ===============================

# 경로 설정
csv_path = "/content/drive/MyDrive/Colab Notebooks/APID/mimic-cxr/llm_train.csv"
image_dir = "/content/drive/MyDrive/Colab Notebooks/APID/mimic-cxr/train"
output_dir = "/content/drive/MyDrive/Colab Notebooks/APID/annotations"

val_json = "/content/drive/MyDrive/Colab Notebooks/APID/mimic-cxr/valid.json"
test_json = "/content/drive/MyDrive/Colab Notebooks/APID/mimic-cxr/test.json"

# 1단계: train_XX.json 생성
convert_partial_csv_to_json(
    csv_path=csv_path,
    image_dir=image_dir,
    output_dir=output_dir,
    ratios=[0.25, 0.5, 0.75, 1.0],
    split_name="train"
)

# 2단계: combined_XX.json 생성
ratios = [25, 50, 75, 100]
for ratio in ratios:
    train_json = os.path.join(output_dir, f"train_{ratio}.json")
    save_path = os.path.join(output_dir, f"combined_{ratio}.json")
    combine_split_jsons(train_json, val_json, test_json, save_path)


총 데이터 수: 83642개
✅ 저장 완료: /content/drive/MyDrive/Colab Notebooks/APID/annotations/train_25.json (20910개 샘플)
✅ 저장 완료: /content/drive/MyDrive/Colab Notebooks/APID/annotations/train_50.json (41821개 샘플)
✅ 저장 완료: /content/drive/MyDrive/Colab Notebooks/APID/annotations/train_75.json (62731개 샘플)
✅ 저장 완료: /content/drive/MyDrive/Colab Notebooks/APID/annotations/train_100.json (83642개 샘플)
✅ 병합 완료: /content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_25.json (train: 20910, val: 710, test: 1446)
✅ 병합 완료: /content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_50.json (train: 41821, val: 710, test: 1446)
✅ 병합 완료: /content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_75.json (train: 62731, val: 710, test: 1446)
✅ 병합 완료: /content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_100.json (train: 83642, val: 710, test: 1446)


# **모델 학습 코드**
  --threshold 3  등장 빈도 3 이상인 단어만 포함시킴
  
  --num_workers 4  PyTorch DataLoader에서 데이터를 병렬로 로딩하는 데 사용하는 프로세스 수
  
  --beam_size 3 리포트 생성 시 beam search를 사용할 때, 후보 문장을 몇 개까지 탐색할지 설정

# 25%

In [ ]:
!python main_train.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_25.json" \
  --image_dir "/content/data" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen25" \
  --epochs 15 \
  --batch_size 64 \
  --max_seq_length 100 \
  --threshold 3 \
  --step_size 1 \
  --gamma 0.8 \
  --num_workers 4 \
  --seed 456789 \
  --beam_size 3

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
	epoch          : 1
	train_loss     : 4.449077258416272
	val_BLEU_1     : 0.06767605830853309
	val_BLEU_2     : 0.04135697172435892
	val_BLEU_3     : 0.028280819152414333
	val_BLEU_4     : 0.02003918698185767
	val_METEOR     : 0.051808050563239026
	val_ROUGE_L    : 0.20316899758333815
	test_BLEU_1    : 0.0538735532654454
	test_BLEU_2    : 0.033294439809657926
	test_BLEU_3 

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen25/current_checkpoint.pth" \
     "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen25/model_best.pth"

# 50%

In [ ]:
!python main_train.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_50.json" \
  --image_dir "/content/data" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen50" \
  --epochs 15 \
  --batch_size 64 \
  --max_seq_length 100 \
  --threshold 3 \
  --step_size 1 \
  --gamma 0.8 \
  --num_workers 4 \
  --seed 456789 \
  --beam_size 3

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
	epoch          : 1
	train_loss     : 3.7674246304625765
	val_BLEU_1     : 0.05871059919091062
	val_BLEU_2     : 0.03088810076142832
	val_BLEU_3     : 0.01872094124693251
	val_BLEU_4     : 0.013191307367221565
	val_METEOR     : 0.048626732393302716
	val_ROUGE_L    : 0.1675113411457687
	test_BLEU_1    : 0.046397150442971276
	test_BLEU_2    : 0.024079389354990797
	test_BLEU_

# 75%

In [ ]:
!python main_train.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_75.json" \
  --image_dir "/content/data" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen75" \
  --epochs 15 \
  --batch_size 64 \
  --max_seq_length 100 \
  --threshold 3 \
  --step_size 1 \
  --gamma 0.8 \
  --num_workers 4 \
  --seed 456789 \
  --beam_size 3

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
	epoch          : 1
	train_loss     : 3.47229706353976
	val_BLEU_1     : 0.054956819449406176
	val_BLEU_2     : 0.028243809773276365
	val_BLEU_3     : 0.01668247200220288
	val_BLEU_4     : 0.011274712745242315
	val_METEOR     : 0.04916611481368709
	val_ROUGE_L    : 0.17260386277151332
	test_BLEU_1    : 0.0436423429734013
	test_BLEU_2    : 0.02264225974108009
	test_BLEU_3  

# 100%

In [ ]:
!python main_train.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_100.json" \
  --image_dir "/content/data" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen100" \
  --epochs 15 \
  --batch_size 32 \
  --max_seq_length 100 \
  --threshold 3 \
  --step_size 1 \

  --gamma 0.8 \
  --num_workers 4 \
  --seed 456789 \
  --beam_size 3

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
	epoch          : 1
	train_loss     : 2.9377756645958573
	val_BLEU_1     : 0.22823230196280314
	val_BLEU_2     : 0.12733645778754277
	val_BLEU_3     : 0.07669074149704572
	val_BLEU_4     : 0.05063113626637012
	val_METEOR     : 0.08663816605465996
	val_ROUGE_L    : 0.2197467981922424
	test_BLEU_1    : 0.21714617038087825
	test_BLEU_2    : 0.12062497740164967
	test_BLEU_3   

# **학습된 모델 불러와서 test 하는 코드**

# 25%

In [ ]:
!python main_test.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_25.json" \
  --image_dir "/content/data" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen25" \
  --load "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen25/model_best.pth" \
  --batch_size 16 \
  --max_seq_length 100 \
  --threshold 3 \
  --beam_size 3


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100% 171M/171M [00:00<00:00, 219MB/s]
06/03/2025 11:46:14 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/Colab Notebooks/APID/output_r2gen25/model_best.pth ...
06/03/2025 11:46:32 - INFO - modules.tester -   Start to

# 50%

In [ ]:
!python main_test.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_50.json" \
  --image_dir "/content/drive/MyDrive/Colab Notebooks/APID/mimic-cxr" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen50" \
  --load "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen50/model_best.pth" \
  --batch_size 16 \
  --max_seq_length 100 \
  --threshold 3 \
  --beam_size 3


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
06/03/2025 12:02:42 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/Colab Notebooks/APID/output_r2gen50/model_best.pth ...
06/03/2025 12:03:08 - INFO - modules.tester -   Start to evaluate in the test set.
91it [10:17,  6.79s/it]
{'test_BLEU_1': 0.2411832317783858, 'test_BLEU_2': 0.141152068973193, 'test_BLEU_3': 0.09065091629003022, 'test_BLEU_4': 0

# 75%

In [ ]:
!python main_test.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_75.json" \
  --image_dir "/content/drive/MyDrive/Colab Notebooks/APID/mimic-cxr" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen75" \
  --load "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen75/model_best.pth" \
  --batch_size 16 \
  --max_seq_length 100 \
  --threshold 3 \
  --beam_size 3


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
06/03/2025 12:13:49 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/Colab Notebooks/APID/output_r2gen75/model_best.pth ...
06/03/2025 12:14:12 - INFO - modules.tester -   Start to evaluate in the test set.
91it [10:08,  6.69s/it]
{'test_BLEU_1': 0.2484951772412596, 'test_BLEU_2': 0.14412935527853782, 'test_BLEU_3': 0.09226641391086406, 'test_BLEU_4':

# 100%

In [ ]:
!python main_test.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_100.json" \
  --image_dir "/content/drive/MyDrive/Colab Notebooks/APID/mimic-cxr" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen100" \
  --load "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen100/model_best.pth" \
  --batch_size 16 \
  --max_seq_length 100 \
  --threshold 3 \
  --beam_size 3


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100% 171M/171M [00:00<00:00, 216MB/s]
06/05/2025 03:13:25 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/Colab Notebooks/APID/output_r2gen100/model_best.pth ...
06/05/2025 03:13:38 - INFO - modules.tester -   Start t

# 학습된 모델 불러와서 시각화 하는 코드

# 25%

In [ ]:
!python main_plot.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_25.json" \
  --image_dir "/content/data" \
  --load "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen25/model_best.pth" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen25/plots" \
  --batch_size 1 \
  --beam_size 1 \
  --max_seq_length 100 \
  --threshold 3

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
06/03/2025 11:57:27 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/Colab Notebooks/APID/output_r2gen25/model_best.pth ...
06/03/2025 11:57:29 - INFO - modules.tester -   Start to plot attention weights in the test set.
0it [00:00, ?it/s][ WARN:0@8.753] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_

# 50%

In [ ]:
!python main_plot.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_50.json" \
  --image_dir "/content/drive/MyDrive/Colab Notebooks/APID/mimic-cxr" \
  --load "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen50/model_best.pth" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen50/plots" \
  --batch_size 1 \
  --beam_size 1 \
  --max_seq_length 100 \
  --threshold 3

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
06/03/2025 12:24:43 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/Colab Notebooks/APID/output_r2gen50/model_best.pth ...
06/03/2025 12:24:46 - INFO - modules.tester -   Start to plot attention weights in the test set.
0it [00:00, ?it/s][ WARN:0@7.851] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_

# 75%

In [ ]:
!python main_plot.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_75.json" \
  --image_dir "/content/drive/MyDrive/Colab Notebooks/APID/mimic-cxr" \
  --load "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen75/model_best.pth" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen75/plots" \
  --batch_size 1 \
  --beam_size 1 \
  --max_seq_length 100 \
  --threshold 3

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
06/03/2025 12:25:18 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/Colab Notebooks/APID/output_r2gen75/model_best.pth ...
06/03/2025 12:25:20 - INFO - modules.tester -   Start to plot attention weights in the test set.
0it [00:00, ?it/s][ WARN:0@8.863] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_

# 100%

In [ ]:
!python main_plot.py \
  --dataset_name mimic_cxr \
  --ann_path "/content/drive/MyDrive/Colab Notebooks/APID/annotations/combined_100.json" \
  --image_dir "/content/drive/MyDrive/Colab Notebooks/APID/mimic-cxr" \
  --load "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen100/model_best.pth" \
  --save_dir "/content/drive/MyDrive/Colab Notebooks/APID/output_r2gen100/plots" \
  --batch_size 1 \
  --beam_size 1 \
  --max_seq_length 100 \
  --threshold 3

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
06/05/2025 03:32:15 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/Colab Notebooks/APID/output_r2gen100/model_best.pth ...
06/05/2025 03:32:18 - INFO - modules.tester -   Start to plot attention weights in the test set.
0it [00:00, ?it/s][ WARN:0@11.544] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to C